In [2]:
from pinecone import Pinecone as PineconeClient
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec
from dotenv import dotenv_values
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
import requests
import json
import ray

/home/jandogonzales/Development/anaconda3/envs/trailz/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


connecto to pinecone api

In [3]:
config = dotenv_values(".env")
env_key = config["PINE_CONE_ENV_KEY"]
api_key = config["PINE_CONE_API_KEY"]
openai_api_key=config["OPENAI_API_KEY"]
cohere_api_key = config["COHERE_API_KEY"]
#pc_index = config["INDEX_NAME"]

pinecone client and embeddings

In [4]:
pineCone = PineconeClient(
    api_key=api_key
)

start to load the data

In [5]:
url = "https://huggingface.co/api/datasets/Cohere/wikipedia-22-12-en-embeddings/parquet/default/train"
response = requests.get(url)
input_files = json.loads(response.content)
columns = ['id', 'title', 'text', 'url', 'emb']
ds = ray.data.read_parquet(input_files, columns=columns) 
print("Dataset:")
print(ds)
print("\n")

/home/jandogonzales/Development/anaconda3/envs/trailz/lib/python3.11/site-packages/ray/data/datasource/parquet_datasource.py:242: FutureWarning: Passing 'use_legacy_dataset' is deprecated as of pyarrow 15.0.0 and will be removed in a future version.
  pq_ds = pq.ParquetDataset(


Metadata Fetch Progress 0:   0%|          | 0/42 [00:00<?, ?it/s]

2024-03-06 13:44:10,948	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Parquet Files Sample 0:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset:
Dataset(
   num_blocks=3542,
   num_rows=35167920,
   schema={
      id: int32,
      title: string,
      text: string,
      url: string,
      emb: list<item: float>
   }
)




(ReadParquet->SplitBlocks(2) pid=78380) /home/jandogonzales/Development/anaconda3/envs/trailz/lib/python3.11/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by promote_options='default'.
(ReadParquet->SplitBlocks(2) pid=78380)   return transform_pyarrow.concat(tables)
(ReadParquet->SplitBlocks(2) pid=79116) /home/jandogonzales/Development/anaconda3/envs/trailz/lib/python3.11/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by promote_options='default'.
(ReadParquet->SplitBlocks(2) pid=79116)   return transform_pyarrow.concat(tables)
(raylet) [2024-03-06 14:09:10,873 E 78307 78307] (raylet) node_manager.cc:3024: 16 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 10b7b0c14f00c92c055af9a8ec33a63d7b3462b1753c797f7a6a772c, IP: 192.168.171.61) over the last time period. To see more information about the Workers killed on this node, u

In [6]:
ds.schema()
count = 0
for row in ds.iter_rows():
    print(row)
    count = count + 1
    if count > 0:
        break

2024-03-06 14:01:38,193	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=1879 for stage ReadParquet to satisfy output blocks of size at most DataContext.get_current().target_max_block_size=128.0MiB.
2024-03-06 14:01:38,194	INFO set_read_parallelism.py:122 -- To satisfy the requested parallelism of 1879, each read task output is split into 2 smaller blocks.
2024-03-06 14:01:38,194	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet]
2024-03-06 14:01:38,195	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-03-06 14:01:38,196	INFO streaming_executor.py:115 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().exe

Running 0:   0%|          | 0/253 [00:00<?, ?it/s]

{'id': 0, 'title': 'Deaths in 2022', 'text': 'The following notable deaths occurred in 2022. Names are reported under the date of death, in alphabetical order. A typical entry reports information in the following sequence:', 'url': 'https://en.wikipedia.org/wiki?curid=69407798', 'emb': [0.2865696847438812, -0.03181683272123337, 0.06668472290039062, 0.03292645886540413, -0.008292825892567635, 0.16873490810394287, -0.0008463106933049858, -0.36077880859375, 0.33916592597961426, 0.3886975646018982, -0.41489771008491516, 0.20758016407489777, -0.11468425393104553, 0.3873162567615509, -0.26252424716949463, 0.007096240296959877, 0.24166066944599152, -0.24653761088848114, 0.060873936861753464, 0.23045268654823303, -0.029800616204738617, 0.5721306800842285, -0.051111623644828796, -0.09547730535268784, 0.1097082868218422, -0.059516504406929016, -0.053682904690504074, 0.23981636762619019, -0.33325839042663574, 0.3685816824436188, 0.18456950783729553, -0.05209290236234665, -0.006129484623670578, 0.

create the index for wikipedia data


<br>
indexes = pc.list_indexes().indexes<br>
print("Indexes:")<br>
print(indexes)<br>
print("\n")<br>



<br>
embeddings = CohereEmbeddings(model="multilingual-22-12", cohere_api_key=cohere_api_key)<br>
print("Embeddings:")<br>
print(embeddings)<br>
print("\n")<br>


get the index from existing srcs


<br>
vectorstore = Pinecone.from_existing_index(index_name=pc_index,<br>
        embedding=embeddings)<br>
retriever = vectorstore.as_retriever()<br>
print("Vector store:")<br>
print(vectorstore)<br>
print("\n")<br>
def fetch_wiki_page(id):<br>
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&pageids={id}"<br>
    response = requests.get(url)<br>
    data = response.json()<br>
    page_content = list(data['query']['pages'].values())[0]['extract']<br>
    return page_content<br>
def fetch_url(x):<br>
    urls = [doc.metadata['url'] for doc in x['context']]<br>
    ids = [url.split('=')[-1] for url in urls]<br>
    contents = [fetch_wikipedia_page(id)[:32000] for id in ids]<br>
    return {"context": contents, "question": x["question"]}<br>


RAG prompt

template = 
Answer the question based only on the following context:<br>
    {context}<br>
    Question: {question}<br>
<br>
rompt = ChatPromptTemplate.from_template(template)

RAG model<br>
odel = ChatOpenAI(temperature=0, model="gpt-4-1106-preview", openai_api_key=openai_api_key)


<br>
chain = (<br>
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})<br>
    | RunnableLambda(fetch_url)<br>
    | prompt<br>
    | model<br>
    | StrOutputParser()<br>
)<br>


In [7]:
# upsert the data
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec

pc = PineconeGRPC()
index_name = 'cohere-wikipedia'

# ensure index DNE
indexes = pc.list_indexes().indexes
names = [_['name'] for _ in indexes]
print("Names:")
print(names)
print("\n")

Names:
['trailz-ai', 'cohere-wikipedia', 'canopy--trailz-ai']




In [8]:
# insert the new wiki index
if index_name not in names:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-west-2'))


In [9]:
import numpy as np
from retry import retry
from tqdm.auto import tqdm

# process the data into vector format
def create_pc_dataset(dataset):
    pcDataList = []
    for row in dataset.iter_rows():
        newRow = {}
        newRow['_id'] = row['id']
        newRow['metadata'] = {}
        newRow['metadata']['title'] = row['title']
        newRow['metadata']['text'] = row['text']
        newRow['vector'] = row['emb']
        pcDataList.append(newRow)
    return pcDataList
        
def upload_batches(dataset):
    batch_size=350

# insert records async
def upload(batch):
    client = PineconeGRPC()
    index = client.Index(index_name)

    # sets the returned and error vectors
    total_vectors = 0
    num_failures = 0

    # data = process_data(large_batch).to_dict(orient='records')
    data = batch.to_dict(orient='records')

    # this will retry up to 2 times, exponential wait increase from min to 4-10
    @retry(stop=stop_after_attempt(2), wait=wait_exponential(multiplier=1, min=4, max=10))
    def send_batch(batch):
        return index.upsert(vectors=batch)

    try:
        result = send_batch(data)
        total_vectors += result.upserted_count
    except Exception as e:
        logging.exception(e)
        num_failures += len(data)
    return {'upserted': np.array([total_vectors]), 'errors': np.array([num_failures])}

class Upserter:
    def __call__(self, large_batch):
        return upload_batches(large_batch)

In [13]:
# set the env var for ray memory error
%env RAY_memory_monitor_refresh_ms=0

env: RAY_memory_monitor_refresh_ms=0


In [15]:
# run the upload batches
from datetime import datetime
from datasets import Dataset
import pickle

%env RAY_memory_monitor_refresh_ms=0

# create the new dataset
pcDataList = create_pc_dataset(ds)
pcDataset = Dataset.from_list(pcDataList)

# create a pkl file for use later so we don't keep reloading
pkl_data = 'pkl_data'
with open(pkl_data+'/pc_dataset.pkl', 'wb') as f:
    pickle.dump(pcDataset, f)

# upload the new ds in batches

# new_ds = ds.map_batches(
#     Upserter,
#     batch_size=batch_size,
#     batch_format='pandas',
#     zero_copy_batch=True,
#     concurrency=1)

# before = datetime.now()
# summary = new_ds.materialize().sum(['upserted', 'errors'])

# summary
# duration = datetime.now() - before
# print({k: f"{v: ,}" for k,v in summary.items()})
# print(f"Duration = {duration}")

2024-03-06 14:16:54,649	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=1879 for stage ReadParquet to satisfy output blocks of size at most DataContext.get_current().target_max_block_size=128.0MiB.
2024-03-06 14:16:54,650	INFO set_read_parallelism.py:122 -- To satisfy the requested parallelism of 1879, each read task output is split into 2 smaller blocks.
2024-03-06 14:16:54,650	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet]
2024-03-06 14:16:54,651	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-03-06 14:16:54,652	INFO streaming_executor.py:115 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().exe

env: RAY_memory_monitor_refresh_ms=0


Running 0:   0%|          | 0/253 [00:00<?, ?it/s]

2024-03-06 14:16:57,560	ERROR streaming_executor_state.py:496 -- An exception was raised from a task of operator "ReadParquet->SplitBlocks(2)". Dataset execution will now abort. To ignore this exception and continue, set DataContext.max_errored_blocks.


OutOfMemoryError: Task was killed due to the node running low on memory.
Memory on the node (IP: 192.168.171.61, ID: 10b7b0c14f00c92c055af9a8ec33a63d7b3462b1753c797f7a6a772c) where the task (task ID: 7c4e8f70143305f87a049962c9c8200d358c85ee01000000, name=ReadParquet->SplitBlocks(2), pid=92481, memory used=0.11GB) was running was 29.71GB / 31.07GB (0.95626), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: 4a8e175254fb452f0a7ef76e1ef486ab1d3961fd7742e7fd8e8acc99) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 192.168.171.61`. To see the logs of the worker, use `ray logs worker-4a8e175254fb452f0a7ef76e1ef486ab1d3961fd7742e7fd8e8acc99*out -ip 192.168.171.61. Top 10 memory users:
PID	MEM(GB)	COMMAND
77966	14.95	/home/jandogonzales/Development/anaconda3/envs/trailz/bin/python -m ipykernel_launcher -f /home/jand...
3438	1.11	io.elementary.appcenter
5036	0.66	/opt/google/chrome